In [284]:
from pyspark.sql import SparkSession 
import pyspark.sql.functions as F

   
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("MigrationExample") \
    .getOrCreate()

# Données d'exemple

matrix = [
    [' Ricky', 4879320143, '2022-09-05 11:23:59', True],
    ['Simon', 4879320143, '2022-09-05 11:23:59', True],
    ['Sergio', 4879320143, '2022-09-05 11:23:59', True],
    ['Pedrito', 4879320143, '2022-09-07 11:23:59', True],
    ['jJuenito', 4879320143, '2022-09-08 11:23:59', True],
    ['Pepe', 4879320143, '2022-09-09 11:23:59', True],
    ['Liusa', 4879320143, '2022-09-05 11:23:59', True],
]

columns = ['name', 'dni', 'date', 'is_human']

spark_df = spark.createDataFrame(matrix, columns)


In [285]:
spark_df.show()

+--------+----------+-------------------+--------+
|    name|       dni|               date|is_human|
+--------+----------+-------------------+--------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|
|   Simon|4879320143|2022-09-05 11:23:59|    true|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|
|    Pepe|4879320143|2022-09-09 11:23:59|    true|
|   Liusa|4879320143|2022-09-05 11:23:59|    true|
+--------+----------+-------------------+--------+



### 1. `lit`: Ajouter une colonne avec une valeur constante

In [286]:
spark_df = spark_df.withColumn("constant", F.lit(1))
spark_df.show()

+--------+----------+-------------------+--------+--------+
|    name|       dni|               date|is_human|constant|
+--------+----------+-------------------+--------+--------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|       1|
|   Simon|4879320143|2022-09-05 11:23:59|    true|       1|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|       1|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|       1|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|       1|
|    Pepe|4879320143|2022-09-09 11:23:59|    true|       1|
|   Liusa|4879320143|2022-09-05 11:23:59|    true|       1|
+--------+----------+-------------------+--------+--------+



### 2. `col` et `when`: Créer une colonne avec une condition

In [287]:
spark_df = spark_df.withColumn("filter", F.when(F.col('name') == "Simon", 1).otherwise(0))
spark_df.show()

+--------+----------+-------------------+--------+--------+------+
|    name|       dni|               date|is_human|constant|filter|
+--------+----------+-------------------+--------+--------+------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0|
|   Simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|       1|     0|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|       1|     0|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|       1|     0|
|    Pepe|4879320143|2022-09-09 11:23:59|    true|       1|     0|
|   Liusa|4879320143|2022-09-05 11:23:59|    true|       1|     0|
+--------+----------+-------------------+--------+--------+------+



### 3. `concat`: Combiner des colonnes en une seule chaîne

In [288]:
spark_df = spark_df.withColumn("concat", F.concat(F.col('name'), F.col('date')))
spark_df.show()

+--------+----------+-------------------+--------+--------+------+--------------------+
|    name|       dni|               date|is_human|constant|filter|              concat|
+--------+----------+-------------------+--------+--------+------+--------------------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|
|   Simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|       1|     0|Sergio2022-09-05 ...|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|       1|     0|Pedrito2022-09-07...|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|       1|     0|jJuenito2022-09-0...|
|    Pepe|4879320143|2022-09-09 11:23:59|    true|       1|     0|Pepe2022-09-09 11...|
|   Liusa|4879320143|2022-09-05 11:23:59|    true|       1|     0|Liusa2022-09-05 1...|
+--------+----------+-------------------+--------+--------+------+--------------------+



#### 4. `month`, `year`, `quarter`: Extraire le mois, l'année et le trimestre

In [289]:
spark_df = spark_df.withColumn("month", F.date_format('date', 'MMMM')) \
    .withColumn("year", F.date_format('date', 'y')) \
    .withColumn("quarter", F.date_format('date', 'QQQQ'))

spark_df.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|
|   Simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|       1|     0|Sergio2022-09-05 ...|September|2022|3rd quarter|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|       1|     0|Pedrito2022-09-07...|September|2022|3rd quarter|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|       1|     0|jJuenito2022-09-0...|September|2022|3rd quarter|
|    Pepe|4879320143|2022-09-09 11:23:59|    true|       1|     0|Pepe2022-09-09

### # 5. `abs` et `floor`: Appliquer des fonctions mathématiques

In [290]:
spark_df = spark_df.withColumn("float", F.lit('-1.12'))
spark_df.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|
|   Simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|       1|     0|Sergio2022-09-05 ...|September|2022|3rd quarter|-1.12|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|       1|     0|Pedrito2022-09-07...|September|2022|3rd quarter|-1.12|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|       1|     0|jJuenito2022-09-0...|September|2022|3rd quarter|-1.12|
|    Pepe|4879320143|2022-09-09 

In [291]:
spark_df = spark_df.withColumn("abs", F.abs(spark_df.float))
spark_df.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|
|   Simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|       1|     0|Sergio2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|       1|     0|Pedrito2022-09-07...|September|2022|3rd quarter|-1.12|1.12|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|       1|     0|jJuenito2022-09-0...|September|2022|3rd quarter|-1.1

In [292]:
spark_df = spark_df.withColumn('floor', F.floor(spark_df.float))
spark_df.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|
|   Simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|       1|     0|Sergio2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|       1|     0|Pedrito2022-09-07...|September|2022|3rd quarter|-1.12|1.12|   -2|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|       1|     0|jJuenito20

In [293]:
spark_df = spark_df.withColumn('ceil', F.ceil(spark_df.float))
spark_df.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1|
|   Simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|       1|     0|Sergio2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|       1|     0|Pedrito2022-09-07...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1|
|jJuenito|4879320143|2022-09-08 11:23:59|

### # 6. `concat_ws`: Concaténer avec un séparateur

In [294]:
liste = [spark_df.name, spark_df.dni]

spark_df = spark_df.withColumn('concat_ws', F.concat_ws(" :) ", *liste))
spark_df.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|
|   Simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :) 4879320143|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|       1|     0|Sergio2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1|Sergio :) 4879320143|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|  

### 7. `array_distinct`, `collect_list`, `collect_set`: Utiliser des fonctions d'agrégation sur des arrays

In [295]:
spark_df.agg(F.array_distinct(F.collect_list('dni'))).show()

+---------------------------------+
|array_distinct(collect_list(dni))|
+---------------------------------+
|                     [4879320143]|
+---------------------------------+



In [296]:
spark_df.agg(F.collect_list('name')).show()

+--------------------+
|  collect_list(name)|
+--------------------+
|[ Ricky, Simon, S...|
+--------------------+



### # 8. `split` et `trim`: Nettoyer et diviser des chaînes

In [297]:
spark_df = spark_df.withColumn('split', F.split('date', " ").getItem(0))
spark_df.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05|
|   Simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :) 4879320143|2022-09-05|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|       1|     0|Sergio2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1|Sergio :) 487932014

In [298]:
spark_df = spark_df.withColumn("a_trim", F.lit("  m p  "))
spark_df.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+-------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split| a_trim|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+-------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05|  m p  |
|   Simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :) 4879320143|2022-09-05|  m p  |
|  Sergio|4879320143|2022-09-05 11:23:59|    true|       1|     0|Sergio2022-09-05 ...|September|2022|3rd quarter|-

In [299]:
spark_df = spark_df.withColumn('trim', F.trim('a_trim'))
spark_df.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+-------+----+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split| a_trim|trim|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+-------+----+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05|  m p  | m p|
|   Simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :) 4879320143|2022-09-05|  m p  | m p|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|       1|     0|Sergio2022-09-05 ...|Sep

In [300]:
spark_df = spark_df.withColumn('name', F.trim('name'))
spark_df.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+-------+----+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split| a_trim|trim|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+-------+----+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05|  m p  | m p|
|   Simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :) 4879320143|2022-09-05|  m p  | m p|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|       1|     0|Sergio2022-09-05 ...|Sep

### 9. `explode`: Diviser une colonne de tableau en lignes distinctes

In [301]:
spark_df = spark_df.withColumn("array name", F.array("name"))
spark_df.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+-------+----+----------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split| a_trim|trim|array name|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+-------+----+----------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05|  m p  | m p|   [Ricky]|
|   Simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :) 4879320143|2022-09-05|  m p  | m p|   [Simon]|
|  Sergio|4879320143|2022-09-05 11:

In [302]:
spark_df = spark_df.withColumn("explode", F.explode("array name"))
spark_df.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+-------+----+----------+--------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split| a_trim|trim|array name| explode|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+-------+----+----------+--------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05|  m p  | m p|   [Ricky]|   Ricky|
|   Simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :) 4879320143|2022-09-05|  m p  | m p|   [Simon]|

### 10. `lower`: Convertir des chaînes en minuscules

In [303]:
spark_df = spark_df.withColumn("name", F.lower(F.col("name")))
spark_df.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+-------+----+----------+--------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split| a_trim|trim|array name| explode|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+-------+----+----------+--------+
|   ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05|  m p  | m p|   [Ricky]|   Ricky|
|   simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :) 4879320143|2022-09-05|  m p  | m p|   [Simon]|

### 11. `select`: Sélectionner des colonnes spécifiques

In [304]:
spark_df.select(F.col('name'), F.col('dni')).show()

+--------+----------+
|    name|       dni|
+--------+----------+
|   ricky|4879320143|
|   simon|4879320143|
|  sergio|4879320143|
| pedrito|4879320143|
|jjuenito|4879320143|
|    pepe|4879320143|
|   liusa|4879320143|
+--------+----------+



### 12. `withColumn`: Créer une nouvelle colonne

In [305]:
spark_df = spark_df.withColumn("name_upper", F.upper(F.col("name")))
spark_df.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+-------+----+----------+--------+----------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split| a_trim|trim|array name| explode|name_upper|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+-------+----+----------+--------+----------+
|   ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05|  m p  | m p|   [Ricky]|   Ricky|     RICKY|
|   simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :) 48

### 13. `withColumnRenamed`: Renommer une colonne

In [306]:
spark_df = spark_df.withColumnRenamed('name', 'NAME')
spark_df.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+-------+----+----------+--------+----------+
|    NAME|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split| a_trim|trim|array name| explode|name_upper|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+-------+----+----------+--------+----------+
|   ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05|  m p  | m p|   [Ricky]|   Ricky|     RICKY|
|   simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :) 48

In [307]:
spark_df = spark_df.withColumnRenamed('NAME', 'name')
spark_df.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+-------+----+----------+--------+----------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split| a_trim|trim|array name| explode|name_upper|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+-------+----+----------+--------+----------+
|   ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05|  m p  | m p|   [Ricky]|   Ricky|     RICKY|
|   simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :) 48

### 14. `drop`: Supprimer une colonne

In [308]:
spark_df = spark_df.drop(spark_df.a_trim)
spark_df.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split|trim|array name| explode|name_upper|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+
|   ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05| m p|   [Ricky]|   Ricky|     RICKY|
|   simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :) 4879320143|2022-09-05| m p|   [Sim

### 15. `join`: Joindre deux DataFrames (ici nous réutilisons le DataFrame lui-même pour l'exemple)

In [309]:
spark_join = spark_df.join(spark_df, spark_df.name == spark_df.name, how="left")
spark_join.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split|trim|array name| explode|name_upper|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split|trim|array name| explode|name_upper|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+

### 16. `groupBy` et `agg`: Grouper par une colonne et appliquer une agrégation

In [310]:
spark_agg = spark_df.groupBy('float').agg(F.mean('float'))
spark_agg.show()

+-----+----------+
|float|avg(float)|
+-----+----------+
|-1.12|     -1.12|
+-----+----------+



### 17. `drop_duplicates`: Supprimer les doublons

In [311]:
spark_duplicate = spark_df.union(spark_df)
spark_duplicate.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split|trim|array name| explode|name_upper|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+
|   ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05| m p|   [Ricky]|   Ricky|     RICKY|
|   simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :) 4879320143|2022-09-05| m p|   [Sim

In [312]:
spark_plus_duplicate = spark_duplicate.drop_duplicates()
spark_plus_duplicate.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split|trim|array name| explode|name_upper|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+
|   ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05| m p|   [Ricky]|   Ricky|     RICKY|
|   simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :) 4879320143|2022-09-05| m p|   [Sim

### 18. `where`/`filter`: Filtrer les lignes selon une condition

In [313]:
spark_df \
    .where(F.col('name') == 'pedrito') \
    .show()

+-------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+-------+----------+
|   name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split|trim|array name|explode|name_upper|
+-------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+-------+----------+
|pedrito|4879320143|2022-09-07 11:23:59|    true|       1|     0|Pedrito2022-09-07...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1|Pedrito :) 487932...|2022-09-07| m p| [Pedrito]|Pedrito|   PEDRITO|
+-------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+-----

### 19. `when` et `otherwise`: Créer une colonne conditionnelle

In [314]:
spark_df \
    .withColumn('when', 
        F.when(F.col('name') == 'jjuenito', 'pouet') \
    .otherwise('no_pouet')) \
    .show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+--------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split|trim|array name| explode|name_upper|    when|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+--------+
|   ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05| m p|   [Ricky]|   Ricky|     RICKY|no_pouet|
|   simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :

### 20. `astype`: Changer le type de colonne (équivalent à `cast`)

In [315]:
spark_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- dni: long (nullable = true)
 |-- date: string (nullable = true)
 |-- is_human: boolean (nullable = true)
 |-- constant: integer (nullable = false)
 |-- filter: integer (nullable = false)
 |-- concat: string (nullable = true)
 |-- month: string (nullable = true)
 |-- year: string (nullable = true)
 |-- quarter: string (nullable = true)
 |-- float: string (nullable = false)
 |-- abs: double (nullable = true)
 |-- floor: long (nullable = true)
 |-- ceil: long (nullable = true)
 |-- concat_ws: string (nullable = false)
 |-- split: string (nullable = true)
 |-- trim: string (nullable = false)
 |-- array name: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- explode: string (nullable = true)
 |-- name_upper: string (nullable = true)



In [316]:
spark_df = spark_df.withColumn('constant', spark_df.constant.astype('string'))

In [317]:
spark_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- dni: long (nullable = true)
 |-- date: string (nullable = true)
 |-- is_human: boolean (nullable = true)
 |-- constant: string (nullable = false)
 |-- filter: integer (nullable = false)
 |-- concat: string (nullable = true)
 |-- month: string (nullable = true)
 |-- year: string (nullable = true)
 |-- quarter: string (nullable = true)
 |-- float: string (nullable = false)
 |-- abs: double (nullable = true)
 |-- floor: long (nullable = true)
 |-- ceil: long (nullable = true)
 |-- concat_ws: string (nullable = false)
 |-- split: string (nullable = true)
 |-- trim: string (nullable = false)
 |-- array name: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- explode: string (nullable = true)
 |-- name_upper: string (nullable = true)



### 21. `isin`: Vérifier si une valeur est dans une liste

In [318]:
spark_df.select('name').show()

+--------+
|    name|
+--------+
|   ricky|
|   simon|
|  sergio|
| pedrito|
|jjuenito|
|    pepe|
|   liusa|
+--------+



In [319]:
spark_df.where(spark_df.name.isin(['simon', 'sergio'])).show()

+------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+-------+----------+
|  name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split|trim|array name|explode|name_upper|
+------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+-------+----------+
| simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :) 4879320143|2022-09-05| m p|   [Simon]|  Simon|     SIMON|
|sergio|4879320143|2022-09-05 11:23:59|    true|       1|     0|Sergio2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1|Sergio :) 4879320143|2022-09-05| m p|  [Sergio]| Sergio|  

### 22. `lower`: Mettre des chaînes en minuscules (déjà utilisé mais réutilisé ici)

In [320]:
spark_df.withColumn("concat_lower", F.lower('concat_ws')).show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+--------------------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split|trim|array name| explode|name_upper|        concat_lower|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+--------------------+
|   ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05| m p|   [Ricky]|   Ricky|     RICKY| ricky :) 4879320143|
|   simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September

### 23. `alias`: Utiliser un alias pour une colonne

In [321]:
spark_df.select(F.col("name").alias('pouet')).show()

+--------+
|   pouet|
+--------+
|   ricky|
|   simon|
|  sergio|
| pedrito|
|jjuenito|
|    pepe|
|   liusa|
+--------+



### 24. `isNull` et `isNotNull`: Vérifier si une colonne est nulle ou non

In [322]:
spark_df.filter(F.col('name').isNotNull()).show()
spark_df.filter(F.col('name').isNull()).show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split|trim|array name| explode|name_upper|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+
|   ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05| m p|   [Ricky]|   Ricky|     RICKY|
|   simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :) 4879320143|2022-09-05| m p|   [Sim

### 25. `like` et `rlike`: Filtrer les chaînes avec un modèle

In [323]:
spark_df.filter(F.col('dni').like('%93%')).show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split|trim|array name| explode|name_upper|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+
|   ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05| m p|   [Ricky]|   Ricky|     RICKY|
|   simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :) 4879320143|2022-09-05| m p|   [Sim

In [324]:
spark_df.filter(F.col('dni').rlike("^[0-9]*$")).show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split|trim|array name| explode|name_upper|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+
|   ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05| m p|   [Ricky]|   Ricky|     RICKY|
|   simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :) 4879320143|2022-09-05| m p|   [Sim

### 26. `regexp_replace`: Remplacer des motifs dans une chaîne avec des expressions régulières

In [325]:
spark_df.select(F.regexp_replace('quarter', r'(\d+)', '--').alias("quarter_replaced")).show()

+----------------+
|quarter_replaced|
+----------------+
|    --rd quarter|
|    --rd quarter|
|    --rd quarter|
|    --rd quarter|
|    --rd quarter|
|    --rd quarter|
|    --rd quarter|
+----------------+



In [326]:
spark_df.show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split|trim|array name| explode|name_upper|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+
|   ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05| m p|   [Ricky]|   Ricky|     RICKY|
|   simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :) 4879320143|2022-09-05| m p|   [Sim

### 27. `str.contains`: Vérifier si une chaîne contient un motif (équivalent à `like`)


In [328]:
spark_df.where(spark_df.quarter.contains('quarter')).show()

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split|trim|array name| explode|name_upper|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+
|   ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05| m p|   [Ricky]|   Ricky|     RICKY|
|   simon|4879320143|2022-09-05 11:23:59|    true|       1|     1|Simon2022-09-05 1...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Simon :) 4879320143|2022-09-05| m p|   [Sim

### differences performances entre collect() - show()

In [329]:
import time

start_time= time.time()
spark_df.where((F.col('concat_ws').like('%Ricky :) 487932014%')) | (F.col('name').like('%jjuenito%'))).show()
end_time = time.time()
duration = end_time - start_time

print(f"Excécution : {duration}")

+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+
|    name|       dni|               date|is_human|constant|filter|              concat|    month|year|    quarter|float| abs|floor|ceil|           concat_ws|     split|trim|array name| explode|name_upper|
+--------+----------+-------------------+--------+--------+------+--------------------+---------+----+-----------+-----+----+-----+----+--------------------+----------+----+----------+--------+----------+
|   ricky|4879320143|2022-09-05 11:23:59|    true|       1|     0| Ricky2022-09-05 ...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1| Ricky :) 4879320143|2022-09-05| m p|   [Ricky]|   Ricky|     RICKY|
|jjuenito|4879320143|2022-09-08 11:23:59|    true|       1|     0|jJuenito2022-09-0...|September|2022|3rd quarter|-1.12|1.12|   -2|  -1|jJuenito :) 48793...|2022-09-08| m p|[jJueni

In [330]:
start_time= time.time()
spark_df.where((F.col('concat_ws').like('%Ricky :) 487932014%')) | (F.col('name').like('%jjuenito%'))).collect()
end_time = time.time()
duration = end_time - start_time

print(f"Excécution : {duration}")


Excécution : 0.2285163402557373


### differences performances entre RDD - count()

In [335]:
start_time= time.time()
spark_df.count()
end_time = time.time()
duration = end_time - start_time

print(f"Excécution : {duration}")

Excécution : 0.20506024360656738


In [336]:
rdd = spark_df.rdd

start_time= time.time()
rdd.count()
end_time = time.time()
duration = end_time - start_time

print(f"Excécution : {duration}")

Excécution : 0.351226806640625
